In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
cities = pd.read_csv('../output_data/weathercsv.csv')
cities.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.59,26.89,59.67,34,0,9.48,ZA,1594600970
1,1,jiuquan,39.74,98.52,84.92,23,100,10.87,CN,1594601138
2,2,beringovskiy,63.05,179.32,51.19,81,100,9.42,RU,1594601042
3,3,gambela,8.25,34.58,69.64,90,100,1.99,ET,1594601164
4,4,punta arenas,-53.15,-70.92,33.80,83,24,8.05,CL,1594601164


In [3]:
humidity = cities['Humidity'].astype(float)
maxhumidity = humidity.max()
locations = cities[['Latitude', 'Longitude']]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
plt.savefig('../output_data/humidgmap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [4]:

newdf = cities.loc[(cities['Max Temp'] > 70) & (cities['Max Temp'] < 95) & (cities['Cloudiness'] == 0) & (cities['Wind Speed'] < 10), :]
newdf = newdf.dropna(how='any')
newdf.reset_index(inplace=True)
del newdf['index']
del newdf['Unnamed: 0']
newdf

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushtobe,45.25,77.98,70.65,36,0,4.14,KZ,1594601183
1,ilhabela,-23.78,-45.36,74.70,61,0,5.84,BR,1594601187
2,agropoli,40.35,15.00,73.00,62,0,5.01,IT,1594601197
3,la solana,38.94,-3.24,73.00,78,0,8.01,ES,1594601198
4,mitsamiouli,-11.38,43.28,77.31,73,0,4.74,KM,1594601209
5,aksu,41.12,80.26,74.41,32,0,2.64,CN,1594601210
6,castelo branco,39.82,-7.49,80.01,42,0,1.99,PT,1594601215
7,cascais,38.70,-9.42,73.99,53,0,1.99,PT,1594601220
8,nola,40.92,14.53,77.00,44,0,9.17,IT,1594601223
9,sao jose da coroa grande,-8.90,-35.15,74.75,79,0,8.55,BR,1594601236


In [5]:
hotel_name = []

# params dictionary to update each iteration
params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key}

for index, row in newdf.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()

    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
newdf['Hotel Name']=hotel_name
newdf = newdf.dropna(how='any')
newdf

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,ilhabela,-23.78,-45.36,74.70,61,0,5.84,BR,1594601187,Ilha Flat Hotel
2,agropoli,40.35,15.00,73.00,62,0,5.01,IT,1594601197,Paestum Inn Beach Resort
3,la solana,38.94,-3.24,73.00,78,0,8.01,ES,1594601198,Hotel El Lagar
4,mitsamiouli,-11.38,43.28,77.31,73,0,4.74,KM,1594601209,Moina Lisa
5,aksu,41.12,80.26,74.41,32,0,2.64,CN,1594601210,Tianyuan International Hotel
6,castelo branco,39.82,-7.49,80.01,42,0,1.99,PT,1594601215,MELIÃ Castelo Branco
7,cascais,38.70,-9.42,73.99,53,0,1.99,PT,1594601220,Hotel Albatroz Cascais
8,nola,40.92,14.53,77.00,44,0,9.17,IT,1594601223,Holiday Inn Nola - Naples Vulcano Buono
9,sao jose da coroa grande,-8.90,-35.15,74.75,79,0,8.55,BR,1594601236,Pousada Barra Velha
11,aljezur,37.32,-8.80,75.99,51,0,6.53,PT,1594601253,Vicentina Aparthotel


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in newdf.iterrows()]
locations = newdf[["Latitude", "Longitude"]]

In [8]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))